# Kees thesis prototype
## implementing An Adaptive Large Neighborhood Search Heuristic for the Pickup and Delivery Problem with Time Windows 
source: [Stefan Ropke, David Pisinger, 2005](http://orbit.dtu.dk/fedora/objects/orbit:8676/datastreams/file_3154899/content)



## import test data ropke and pisinger used


### select test instance

In [1]:
import os
PATH_INSTANCES = "/home/sandralaptop/kees/thesis/git_dir/prototypes/pdptw/data/PDPTWInstancesRopke/"
list_files = os.listdir(PATH_INSTANCES)
list_instances = []
INSTANCE_PREFIX = 'prob'
for filename in list_files:
    if filename[0:4] == INSTANCE_PREFIX:
        list_instances.append(filename)
list_instances.sort()

In [2]:
import ipywidgets as widgets
from IPython.display import display
widget_filepicker = widgets.Dropdown(
    options=list_instances,
    value='prob50A.txt',
    description='file')
display(widget_filepicker)

## transform raw file
I read the txt file first to differnt lists.
Then transform the lists to pandas dataframes and in servicable case to dictionary


LESSON!

    df1= pd.read_table("data/PDPTWInstancesRopke/prob50A.txt",
              header=None, skiprows=1)

In [3]:
PATH_TO_INSTANCE = PATH_INSTANCES+widget_filepicker.value
import pandas as pd

def read_args(intlist):
    list_parameter_keys = ['NUMBER_OF_REQUESTS',
                      'MAX_NUMBER_VEHICLES',
                      'ALPHA',
                      'BETA',
                      'GAMMA']
    params = dict(zip(list_parameter_keys, intlist)) 
    return params

def rawfile_to_lists(rawfile):    
    listdf_requests = []
    listdf_trucks = []
    list_servable_by = []
    for i, line in enumerate(rawfile):
        list_ints = [int(txt) for txt in line.split("\t")]

        if i == 0:

            params = read_args(list_ints)

        elif i <= 2*params['NUMBER_OF_REQUESTS']:

            listdf_requests.append(list_ints[:10])

            list_servable_by.append(list_ints[9:])
        else:
            listdf_trucks.append(list_ints)

    return params, listdf_requests, listdf_trucks, list_servable_by

def list_to_df_requests(listdf_requests):
    cols = ['node id','x coordinate', 'y coordinate', 'demand', 
        'time window start','time window end','service time',
        'predecessor node id', 'successor node id', 
        'pickup by all']

    df_requests = pd.DataFrame(listdf_requests)
    df_requests.columns = cols    
    #
    #add column for functionality and plotting
    #
    df_requests.loc[(df_requests['pickup by all'] > 0), ['pickup by all']] = 'no'
    df_requests.loc[(df_requests['pickup by all'] == -1), ['pickup by all']] = 'yes'
    df_requests.loc[(df_requests['demand'] < 0), ['pickup by all']] = 'delivery'

    return df_requests

def list_to_df_trucks(listdf_trucks):
    cols = ['vehicle id', 'start terminal x coordinate', 
        'start terminal y coordinate', 'end terminal x coordinate', 
        'end terminal y coordinate', 'vehicle capacity', 
        'vehicle start time', 'vehicle end time' ]

    df_trucks = pd.DataFrame(listdf_trucks)
    df_trucks.columns = cols
    return df_trucks

def transform_list_servicable_to_dict(list_servable_by):
    #key is request, value is list of all trucks that can service request
    dict_servicable = {}
    for index, item in enumerate(list_servable_by):
        if item and item != [-1]:
            dict_servicable[index] = item
    return dict_servicable

def file_transform(path):
    with open(path) as rawfile:
        params, listdf_requests, listdf_trucks, list_servable_by = rawfile_to_lists(rawfile)
        
        df_requests = list_to_df_requests(listdf_requests)
        df_trucks = list_to_df_trucks(listdf_trucks)
        dict_servicable = transform_list_servicable_to_dict(list_servable_by)
        return params, df_requests, df_trucks, dict_servicable 

params, df_requests, df_trucks, dict_servicable = file_transform(PATH_TO_INSTANCE)



## optionally use toy version of instance
fist copy original

In [4]:
import copy
params_orig = copy.deepcopy(params)
df_requests_orig = copy.deepcopy(df_requests)
df_trucks_orig = copy.deepcopy(df_trucks)
dict_servicable_orig = copy.deepcopy(dict_servicable)

choose size to transform instance into

In [5]:
from ipywidgets import interact, interactive, fixed 
from IPython.display import display
import ipywidgets as widgets

def handler(NUMBER_OF_REQUESTS,MAX_NUMBER_VEHICLES):
    print '(NUMBER_OF_REQUESTS = %i , MAX_NUMBER_VEHICLES = %i)' %  (NUMBER_OF_REQUESTS,MAX_NUMBER_VEHICLES) 

def toy_instance_widget():
    
    widget_NUMBER_OF_REQUESTS =widgets.IntSlider(
        min=1,
        max=params_orig['NUMBER_OF_REQUESTS'],
        step=1,
        value=5
        );

    widget_MAX_NUMBER_VEHICLES =widgets.IntSlider(
        min=1,
        max=params_orig['MAX_NUMBER_VEHICLES'],
        step=1,
        value=2
        );

    widget_input = interactive(handler, 
                    NUMBER_OF_REQUESTS=widget_NUMBER_OF_REQUESTS, 
                    MAX_NUMBER_VEHICLES=widget_MAX_NUMBER_VEHICLES)

    return widget_input

widget_input_toy = toy_instance_widget()
display(widget_input_toy)


(NUMBER_OF_REQUESTS = 5 , MAX_NUMBER_VEHICLES = 2)


# Reset instance size
## dataframes to toy instance

In [6]:
params.update(widget_input_toy.kwargs)
df_requests = df_requests_orig.loc[range(2*params['NUMBER_OF_REQUESTS'])]
df_trucks = df_trucks_orig.loc[range(params['MAX_NUMBER_VEHICLES'])]

dict_servicable = {}
for key in dict_servicable_orig.keys():
    if key<2*params['NUMBER_OF_REQUESTS']:
        dict_servicable[key] = dict_servicable_orig[key]

### bokeh

In [7]:
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
import matplotlib
cnames = matplotlib.colors.cnames
output_notebook()
def plot_landscape(df_requests, df_trucks):
    TOOLS = 'pan,box_select,resize,reset, save, wheel_zoom, hover'
    fig = figure(title='landscape', tools=TOOLS)
    source_yes = ColumnDataSource(df_requests.loc[(df_requests['pickup by all'] == 'yes')])
    s1 = fig.scatter('x coordinate','y coordinate',
                     source=source_yes, color=cnames['green'],size=10,legend='pickup by all')
    source_delivery = ColumnDataSource(df_requests.loc[(df_requests['pickup by all'] == 'delivery')])
    s2 = fig.scatter('x coordinate','y coordinate', 
                     source = source_delivery,color=cnames['red'],size=10,legend='delivery')
    source_no = ColumnDataSource(df_requests.loc[(df_requests['pickup by all'] == 'no')])
    s3 = fig.scatter('x coordinate','y coordinate', 
                     source = source_no,color=cnames['black'],size=10,legend='pickup not by all')
    source_trucks = ColumnDataSource(df_trucks)
    s4 = fig.scatter('start terminal x coordinate','start terminal y coordinate',
                     source=source_trucks, color=cnames['blue'],size=10,legend='start')
    s5 = fig.scatter('end terminal x coordinate','end terminal y coordinate', 
                     source = source_trucks,color=cnames['yellow'],size=5,legend='end')
    return fig
fig = plot_landscape(df_requests, df_trucks)
show(fig)

BokehJS successfully loaded.

### calculate location and distance matrix

In [8]:
import numpy as np
def calc_location_array(df_requests, df_trucks, params):
    #set up locations array
    location_array = np.zeros((2*params['NUMBER_OF_REQUESTS'] 
                            + 2*params['MAX_NUMBER_VEHICLES'] ,2))
    #transform location values from datafram to nd array
    requests_locations = df_requests[['x coordinate', 'y coordinate']].values
    trucks_start_locations = df_trucks[['start terminal x coordinate', 'start terminal y coordinate']].values
    trucks_end_locations = df_trucks[['end terminal x coordinate', 'end terminal y coordinate']].values

    #store nd arrays of requests in location array. 
    location_array[:len(requests_locations),:] = requests_locations
    #store ndarrays of trucks locations below request locations in same array
    for i in xrange(params['MAX_NUMBER_VEHICLES']):
        index_start_loc = 2*params['NUMBER_OF_REQUESTS'] + 2*i
        location_array[index_start_loc,:] = trucks_start_locations[i,:]
        index_end_loc = 2*params['NUMBER_OF_REQUESTS'] + 2*i + 1
        location_array[index_end_loc,:] = trucks_end_locations[i,:]
    return location_array

location_array = calc_location_array(df_requests, df_trucks, params)

In [9]:
import scipy.spatial.distance as ds
distance_matrix = ds.squareform(ds.pdist(location_array))



## simplify my life a bit
naming

In [10]:
n_veh = params['MAX_NUMBER_VEHICLES']
n_req = params['NUMBER_OF_REQUESTS']
ALPHA = params['ALPHA']
BETA = params['BETA']
GAMMA = params['GAMMA']

indexing

In [11]:
import numba

def reqid_2_locids( reqid): 
    return reqid*2, reqid*2+1

def vehid_2_locids( vehid):
    return 2*n_req+2*vehid, 2*n_req+2*vehid+1 

# set up the algo
### decision variables

Make a solution object that contains the following decision variabels

- adjacency matrix $X$ ijk : binary varaible which is one if the edge between node i and j is used by vhecile k and 0 otherwise


- Time matrix: S ik  nonnegative integer that indicates when vehcile k start the service at location i
- Load matrix: L ik nongegative integer that is upper bound on the amount of goods on vehicel k after servicing node i
- Request bank : Zi binary variable that indicates if request i is placed in request bank. 1 if in request bank zero otherwise


In [12]:
def init_vars(n_veh, n_req): 
    n_locs = 2*n_req + 2*n_veh
    
    #adjacency matrix
    adj_matrix = np.zeros((n_veh, n_locs, n_locs))
    for truck_i in xrange(n_veh):
        ids = vehid_2_locids(truck_i)
        adj_matrix[truck_i][ids[0]][ids[1]] = 1    
    
    #time table 
    time_table = np.zeros((n_locs,n_veh ))
    # fill in times trucks take to go from start to finish
    for truck_i in xrange(n_veh):
        ids = vehid_2_locids(truck_i)
        time_table[2*n_req+truck_i][truck_i] =  distance_matrix[ids[0]][ids[1]]
            
    #load matrix, all zero
    load_table = np.zeros((n_locs,n_veh ))
    # request bank all are uninserted, hence 1
    request_bank = np.ones(n_req)
       
    return adj_matrix, time_table, load_table, request_bank

adj_matrix, time_table, load_table, request_bank = init_vars(n_veh, n_req)

## to determine how to construct route, we need to know objective value

In [13]:
def calc_distance(adj_matrix,distance_matrix):
    #collapse all trucks into from 3 to 2 dim matrix since costs of all vehicles the same
    edges = adj_matrix.sum(0)
    return distance_matrix[edges.nonzero()].sum()

def calc_duration(time_table):
    # time at end node is latest time, and start always at time 0
    return  time_table.max(1).sum()

def calc_objective_value(distance,duration,n_request_bank):
    objective_value = params['ALPHA'] * distance + params['BETA'] * duration\
             + params['GAMMA'] * n_request_bank
    return objective_value
    
#def calc_objective_value(params, distance,duration,n_request_bank):
#    objective_value = params['ALPHA'] * distance + params['BETA'] * duration\
#             + params['GAMMA'] * n_request_bank
#    return objective_value

now calculate components of objective value and the objective value

In [14]:
distance = calc_distance(adj_matrix,distance_matrix)
duration = calc_duration(time_table)
n_request_bank = request_bank.sum()

objective_value = calc_objective_value(distance,duration,n_request_bank)

# lets plot some stuff

let's make life easier by improving df_requests with a column 'request id' and 'location type' 

In [15]:
df_requests['request id'] = (df_requests['node id']/2).astype('int')

df_requests.loc[df_requests['pickup by all'] == 'delivery', 'location type'] = 'delivery'
df_requests.loc[df_requests['pickup by all'] != 'delivery', 'location type'] = 'pickup'

make better plot with bokeh with boxes

In [16]:
from bokeh.models import Range1d
from bokeh.palettes import Spectral11
p = figure(width=500, height=250, y_range = Range1d(0,5000), title = 'time windows')

for i, loc_type in enumerate(['pickup', 'delivery']):
    source = ColumnDataSource(df_requests.loc[df_requests['location type'] == loc_type])
    p.segment(x0='request id', y0='time window start', x1='request id',
          y1='time window end', line_width=10, color = Spectral11[i],
          source = source, alpha = .4, legend=loc_type)

p.xaxis.axis_label = "request id"
p.yaxis.axis_label = "time"
show(p);


## first an initial route

Lets first insert  request 0 in route 0. otherwhise testing with empty routes.


First fix adj_matrix

In [17]:
req_id = 0
veh_id = 0

#get loc ids
start, end = vehid_2_locids(veh_id)
pickup, delivery = reqid_2_locids(req_id)
#remove edge from start to end
adj_matrix[veh_id,start,end] = 0
#vehicle 0 goes from start loc to pickup to delivery to end loc
from_ls = [start, pickup, delivery]
to_ls = [pickup, delivery, end]
# put this request in route 0

adj_matrix[veh_id,from_ls,to_ls] = 1

time_table 

In [18]:
# set pickup. 
window_start = df_requests['time window start'][pickup]
time_table[pickup][veh_id] = max(distance_matrix[start][pickup], 
                                  window_start)
#set delivery
window_start = df_requests['time window start'][delivery]
time_table[delivery][veh_id] = max(time_table[pickup][veh_id] + 
                                    df_requests['service time'][pickup] + 
                                    distance_matrix[pickup][delivery],
                                    window_start)
                                     
# set end time
time_table[end][veh_id] = time_table[delivery][veh_id] \
                            + df_requests['service time'][delivery] \
                            + distance_matrix[delivery][end]  

load_table

In [19]:
load_table[pickup][veh_id] = df_requests['demand'][pickup]
load_table[delivery][veh_id] = load_table[pickup][veh_id]\
                                + df_requests['demand'][delivery]

request_bank

In [20]:
request_bank[req_id] = 0

## Now see where second request can be put in vehicle 0

In [21]:
req_id = 1
start_req, end_req = reqid_2_locids(req_id)

to know where the requests can be put, I have to know which locations already in vehicle 0

In [22]:
def vehid_2_route(veh_id,time_table):
    start, _ = vehid_2_locids(veh_id) 
    n_req_locs = np.shape(np.nonzero(time_table[:2*n_req,veh_id]))[1] 
    rt_len = n_req_locs + 2
    result = np.zeros((rt_len,), dtype='int')
    result[0] = start
    result[1:] = np.array(np.argsort(time_table[:,veh_id])[-rt_len+1:])
    return result

route = vehid_2_route(veh_id,time_table)
n_pos = len(route) - 1
n_current = len(route)

import some stuff to arrays for fast lookup

In [23]:
veh_cap = df_trucks['vehicle capacity'].values.astype('int')
# req_loads = df_requests.loc[df_requests['location type'] == 'pickup', ['demand']].values
request_demands = np.concatenate(( df_requests['demand'].values, np.zeros([2*n_veh,]) )).astype('int')
service_times = np.concatenate(( df_requests['service time'].values, np.zeros([2*n_veh,]) )).astype('int')
time_windows_start = np.concatenate((df_requests['time window start'].values, np.zeros([2*n_veh,]) )).astype('int')
time_windows_end = np.concatenate(( df_requests['time window end'].values, np.full((2*n_veh,), fill_value = 5000 ))).astype('int')

/home/sandralaptop/anaconda2/lib/python2.7/site-packages/numpy/core/numeric.py:294: FutureWarning: in the future, full((4,), 5000) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


insert new pickup and delivery into route to get new_route

In [24]:
@numba.jit(nopython=True)
def insert_into_route(i,j,pickup,delivery,route):
    new_route = np.empty(route.shape[0]+2)
    new_route[:i] = route[:i]
    new_route[i] = pickup
    new_route[i+1:j] = route[i:j-1]
    new_route[j] = delivery
    new_route[j+1:] = route[j-1:]
    return new_route

from route build adj_matrix, load_table, time_table

In [25]:
np.empty([3])

array([  0.00000000e+000,   4.94065646e-324,   5.43472210e-323])

In [26]:
@numba.jit
def route_2_adjM(route, adjM, veh_id):
    cand_adjM = adjM.copy()
    #for only 1 veh so  cand_adjm = 2dim array 
    cand_adjM[veh_id,:,:] = 0    
    for k in xrange(route.shape[0]-1):
        cand_adjM[veh_id, route[k],route[k+1]] = 1
    return cand_adjM

@numba.jit
def route_2_tt(route, tt, veh_id):
    cand_tt = tt.copy() 
    cand_tt[:,veh_id] = 0 
    
    for k in xrange(route.shape[0]-1):
        frm = route[k]
        to = route[k+1]
        T_arriv = cand_tt[frm,veh_id] + service_times[frm] + distance_matrix[frm,to]
        cand_tt[to,veh_id] = max(T_arriv, time_windows_start[to])      
    return cand_tt

@numba.jit
def route_2_lt(route, lt, veh_id):
    cand_lt = lt.copy()
    cand_lt[:,veh_id] = 0

    for k in xrange(route.shape[0]-1):
        frm = route[k]
        to = route[k+1]
        cand_lt[to, veh_id] =  cand_lt[frm,veh_id] + request_demands[to]  
    return cand_lt      

## validation of candidate. implement constraints from paper
make life first bit easier

In [27]:
PICKUPS = range(0,2*n_req,2)
DELIVERIES = range(1,2*n_req,2)
REQ_LOCS = range(2*n_req)
VEHICLES = range(n_veh)

def Vks(veh_id):
    return REQ_LOCS + list(vehid_2_locids(veh_id))

constraint 2 ensures that each pickup location is visited or that the corresponding request is placed in the
request bank



In [46]:
def constr2(adjM, rb):
    for pickup in PICKUPS:
        if not adjM[:,pickup, REQ_LOCS].sum() + rb[pickup/2] == 1:
            #print 'fail 2'
            return False
    return True

 Constraint (3) ensures that the delivery location is visited if the pickup location is visited and
that the visit is performed by the same vehicle

In [29]:
def constr3(adjM):
    for veh in VEHICLES:
        Vk = Vks(veh)
        for pickup in PICKUPS:
            delivery = pickup+1
            if not np.sum(adjM[veh,pickup,Vk]) - np.sum(adjM[veh, Vk, delivery]) == 0:
                print 'fail 3'
                return False
    return True

 Constraints (4) and (5) ensure that a vehicle leaves every start
terminal and a vehicle enters every end terminal. Together with constraint (6) this ensures that consecutive
paths between τk and τ
0
k
are formed for each k ∈ K

    ∑j∈Pk∪{τ0k}xτk, j,k = 1 ∀k ∈ K (4)
    ∑i∈Dk∪{τk}xi,τ0k,k = 1 ∀k ∈ K (5)
    ∑i∈Vkxi jk − ∑i∈Vk xjik = 0 ∀k ∈ K,∀ j ∈ Nk (6)

In [30]:
def constr4(adjM):
    for k in VEHICLES:
        vehicle_start,vehicle_end = vehid_2_locids(k)
        loc_list = PICKUPS + [vehicle_end]
        if not adjM[k,vehicle_start,loc_list].sum() == 1:
            print 'fail 4'
            return False
    return True

def constr5(adjM):
    for k in VEHICLES:
        vehicle_start,vehicle_end = vehid_2_locids(k)
        loc_list = DELIVERIES + [vehicle_start]
        if not adjM[k,loc_list,vehicle_end].sum() == 1:
            print 'fail 5'
            return False
    return True
    
def constr6(adjM):
    for k in VEHICLES:
        vk = Vks(k)
        for loc in REQ_LOCS:        
            if not adjM[k,vk,loc].sum() - adjM[k,loc,vk].sum() == 0:
                print 'fail 6'
                return false
    return True

Constraints (7), (8) ensure that Sik is set correctly along the paths and that the time windows are obeyed.
These constraints also make sub tours impossible


    xi jk = 1 ⇒ Sik +si +ti j ≤ Sjk ∀k ∈ K,∀(i, j) ∈ Ak     (7)

In [32]:
def constr7(adjM, tt, vehs=VEHICLES):
    """
    use third variable vehs if to indicate which vehicels to check. default = all
    """
    indices = np.transpose(np.nonzero(adjM[vehs,:,:]))
    for veh,pickup,delivery in indices:
        if not (tt[pickup, veh] + service_times[pickup] + distance_matrix[pickup,delivery] \
                                                <= tt[delivery, veh]):
            print 'fail 7'
            return False
    return True               

    ai ≤ Sik ≤ bi ∀k ∈ K,∀i ∈ Vk   (8)

In [33]:

#def constr8(tt):
#    """
#    use third variable vehs if to indicate which vehicels to check. default = all
#    """
#    indices = np.transpose(np.nonzero(tt))
#    for loc,veh in indices:
#        # print 'loc,veh = ', loc, veh
#        if not (time_windows_start[loc] <= tt[loc, veh] <= time_windows_end[loc]):
#        #    print 'fail 8'
#            return False
#    return True        

@numba.jit(nopython=True)
def constr8(tt):
    locs,vehs = np.nonzero(tt)
    for i in range(locs.shape[0]):
        if not (time_windows_start[locs[i]] <= tt[locs[i], vehs[i]] <= time_windows_end[locs[i]]):
            return False
    return True  

Constraint (9) ensures that each pickup occur before the
corresponding delivery
      
      
      Sik ≤ Sn+i,k ∀k ∈ K,∀i ∈ Pk

In [34]:
def constr9(tt):
    for veh in VEHICLES:
        for pickup in PICKUPS:
            delivery = pickup+1
            if not tt[pickup,veh] <= tt[delivery,veh]:
                print 'fail 9'
                return False
    return True

 Constraints (10),(11) and (12) ensure that the load variable is set correctly along the
paths and that the capacity constraints of the vehicles are respected.

    xi jk = 1 ⇒ Lik +lj ≤ Ljk ∀k ∈ K,∀(i, j) ∈ Ak (10)
    Lik ≤ Ck ∀k ∈ K,∀i ∈ Vk (11)
    Lτkk = Lτ0kk = 0 ∀k ∈ K (12)


In [35]:
def constr10(adjM,lt):
    idxs = np.transpose(np.nonzero(adjM))
    for veh, pickup, delivery in idxs:
        if not lt[pickup, veh] + request_demands[delivery] <= lt[delivery,veh]:
            print 'fail 10'
            return False
    return True        

def constr11(lt):
    for veh in VEHICLES:
        for loc in Vks(veh):
            if not lt[loc,veh] <= veh_cap[veh]:
                print 'fail 11'
                return False
    return True

def constr12(lt):
    for veh in VEHICLES:
        start,end = vehid_2_locids(veh)
        if lt[start,veh] or lt[start,veh]:
            print 'fail 12'
            return False
    return True

## check all constraints defined above

In [50]:

def check_feasibility(adjM, tt, lt, rb):
    """
    checks feasiblity 
    """
    if not constr8(tt):
        return False
    if not constr2(adjM,rb):
        return False
    #print 'not failed constr 2'
    if not constr3(adjM):
        return False
    if not constr7(adjM,tt):
        return False
    if not constr4(adjM):
        return False
    if not constr5(adjM):
        return False
    if not constr6(adjM):
        return False
    if not constr9(tt):
        return False
    if not constr10(adjM, lt):
        return False
    if not constr11(lt):
        return False
    if not constr12(lt):
        return False
    return True

copy original decision variables, so that new route can be inserted, without losing current solution

In [37]:
adj_matrix_orig = adj_matrix.copy()
time_table_orig = time_table.copy()
load_table_orig = load_table.copy()
request_bank_orig = request_bank.copy()
route_orig = route.copy()

make new objective value function to make life easier

In [38]:
def calc_ov(adjM, tt, rb) :
    distance = calc_distance(adjM,distance_matrix)
    duration = calc_duration(tt)
    n_request_bank = rb.sum()
    return calc_objective_value(distance,duration,n_request_bank)

where can pickups be inserted? if n current locations, list with pickup will be n+1 in length, the pickup can become position in  1,2,...,n-1, because 0 is for start vehicle, and n is for end location vehicle. delivery can come after pickup with last location being n, since n+1 is for end vehicle

In [70]:

def insert_req(veh_id ,req_id,adj_matrix, time_table, load_table, request_bank):
    route = vehid_2_route(veh_id,time_table)
    n_pos = len(route) - 1
    n_current = len(route)
    cand_rb = request_bank.copy()
    cand_rb[req_id] = 0
    start_req, end_req = reqid_2_locids(req_id)
#    valid_solutions = []
    for i in xrange(1,n_current):
        for j in xrange(i+1,n_current+1): 
# from here: put in function
            new_route = insert_into_route(i,j,start_req, end_req, route)

            cand_adjM = route_2_adjM(new_route, adj_matrix, veh_id)
            cand_tt = route_2_tt(new_route, time_table, veh_id)
            cand_lt = route_2_lt(new_route, load_table, veh_id)

            #validate 
            is_feasible = check_feasibility(cand_adjM, cand_tt, cand_lt, cand_rb)
            if not is_feasible:
                continue
            objValue = calc_ov(cand_adjM, cand_tt, cand_rb)  
# till here: put in fucntion
#            valid_solutions.append([objValue,list(new_route)])
#    print valid_solutions
            
%time insert_req(veh_id ,req_id,adj_matrix, time_table, load_table, request_bank)

CPU times: user 1.97 ms, sys: 3.93 ms, total: 5.9 ms
Wall time: 5.13 ms


/home/sandralaptop/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:15: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/sandralaptop/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:16: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/sandralaptop/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:17: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [69]:
a = range(5)
b = list(a)
a[3] = 1
print b

[0, 1, 2, 3, 4]


In [40]:
stop here

SyntaxError: invalid syntax (<ipython-input-40-f9ef205a8f26>, line 1)

In [ ]:
import line_profiler
import IPython
ip = IPython.get_ipython()
ip.define_magic('lprun', line_profiler.magic_lprun)


In [ ]:
%lprun -f insert_req insert_req(veh_id ,req_id,adj_matrix_orig, time_table_orig, load_table_orig, request_bank_orig)

In [ ]:
%lprun -f check_feasibility check_feasibility(adj_matrix, time_table, load_table, request_bank)

In [ ]:
%lprun -f route_2_tt route_2_tt(route, time_table, veh_id)

In [ ]:
%lprun -f constr6 constr6(adj_matrix)

first for the first entry of the request bank print all the combinations of insertions in the first route.

Now print all the insertion pairs that are possible with load and time

In [ ]:
a = np.zeros((3,3))
a[[0,0,0],[0,1,2]] = 1
print a

### cost matrix per truck
Output:

the costs for:
- for each truck that the request can be inserted into
- for each location in the route the pickup of the request can be inserted into
- for each location in the route the delivery of the request can be inserted into


Input:
- start and end time of route
- distance matrix
-

### cost matrix per truck per request
output:

the costs for
- for each location in the route the pickup of the request can be inserted into
- for each location in the route the delivery of the request can be inserted into

### Given:  route, request, pickup method
-1 check if pickup method possible time and load else continue with next pickup method. 
iterate over delivery methods
output:

the costs for
- for each location in the route the delivery of the request can be inserted into

### Given: route, request, pickup method, delivery method
-1 check if delivery possible with time and load and etc..
-2 remove edges 
-3 place new edges
-4 calculate costs with distance matrix and service times

In [ ]:
%%timeit
total = 0
for x in (0,1,2,3):
        for y in (0,1,2,3):
            if x < y:
                total+=x+y

In [ ]:
def build_cost_matrix():
    reqs_to_insert = np.flatnonzero(request_bank)
    n_insert = len(reqs_to_insert)
    # insertion cost vehicle i uninserted req j
    cost_matr = np.zeros(n_veh, n_insert)
    for truck_i in xrange(n_veh):


        for j, req in enumerate(reqs_to_insert):


        #check whether truck i can service request j
            if req in dict_servicable.keys():
                if truck_i not in dict_servicable[req]:
                    cost_matr[truck_i][j] = np.inf
                    print "truck {0} cannot serve request {1} ".format(truck_i, req)
                    continue

        # can be serviced now caluclate cheapest place to insert and cost
        get_best_insert()

## Determine which insertion should be picked

Check per request what the cheapest insertion is(whcih route, when to pickup, when to deliver)

After inserting recalculate the insertion costs for all request in request bank for the route that the last request was inserted into. Insertion costs into other routes stayed the same.

In [ ]:


def store_solution():
    solutions[iteration] = {
        'adj_matrix': adj_matrix.copy(), 
        'time_table': time_table.copy(),
        'load_table': load_table.copy(),
        'request_bank': request_bank.copy() 
        }









In [ ]:
import numpy as np

In [ ]:
a = np.zeros((5000,5000))
a[1000][1000] = 1
a[1000][2000] = 1
b = a.copy()
%time a[1000][:] = 0
%time b[1000][3000] = 0; b[1000][2000] = 0; b[1000][3000] = 0

In [ ]:
#%lprun -f insert_into_route insert_into_route(1,2,start_req, end_req, route_orig)

In [ ]:
#%lprun -f vehid_2_route vehid_2_route(veh_id,time_table_orig)

In [ ]:
lprun -f route_2_tt route_2_tt(route, time_table, veh_id)

## create object 


## Greedy insert heuristic

In [ ]:
print params